# WaveNet

## 波形接続型音声合成システム
- さまざまな音声の断片を保存しておき、合成する際にそれらを組み合わせて出力する手法。
- 拡張性は高くない。
- あらかじめ音声の断片を保存しておく必要があるため、新しい話者や異なる種類の音声に対応する場合はまずその断片の録音から始めなくてはならない。

## パラメトリック音声合成システム
- 音声合成を行うための特徴量を保持したモデルを作成する手法。
- WaveNet以前の信号処理アプ語リズムの処理結果を用いるのに対し、音声を直接モデリングすることで最高精度の更新に成功。

In [1]:
import datetime
import json
import os
import re
import wave

import keras.backend as K
import numpy as np
import scipy.io.wavfile
import scipy.signal
from keras import layers
from keras import metrics
from keras import objectives
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.engine import Input
from keras.engine import Model
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from sacred import Experiment
from sacred.commands import print_config
from tqdm import tqdm
from time import gmtime, strftime
from keras.callbacks import TensorBoard
from keras.layers.convolutional import Conv1D

In [2]:
def categorical_mean_squared_error(y_true, y_pred):
    """MSE for categorical variables."""
    return K.mean(K.square(K.argmax(y_true, axis=-1) -
                           K.argmax(y_pred, axis=-1)))


class CausalAtrousConvolution1D(Conv1D):
    def __init__(self, filters, kernel_size, init='glorot_uniform', activation=None,
                 padding='valid', strides=1, dilation_rate=1, bias_regularizer=None,
                 activity_regularizer=None, kernel_constraint=None, bias_constraint=None, use_bias=True, causal=False, **kwargs):
        super(CausalAtrousConvolution1D, self).__init__(filters,
                                                        kernel_size=kernel_size,
                                                        strides=strides,
                                                        padding=padding,
                                                        dilation_rate=dilation_rate,
                                                        activation=activation,
                                                        use_bias=use_bias,
                                                        kernel_initializer=init,
                                                        activity_regularizer=activity_regularizer,
                                                        bias_regularizer=bias_regularizer,
                                                        kernel_constraint=kernel_constraint,
                                                        bias_constraint=bias_constraint,
                                                        **kwargs)

        self.causal = causal
        if self.causal and padding != 'valid':
            raise ValueError("Causal mode dictates border_mode=valid.")